In [ ]:
from elevation_aware_ssl.EDA import read_geotiff_image, read_numpy_image, less_cloudy_image
from elevation_aware_ssl.preprocessing import from_array_to_geotiff
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import numpy as np
import os
import sys
import elevation
import re
import shutil
from tqdm import tqdm
from elevation_aware_ssl import EDA
import multiprocessing as mp

In [ ]:
path_to_data = "/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset" 

In [ ]:
metadata = pd.read_csv("/media/omar/storage/gdrive/Maestria/Theses/Preprocessing/metadata_raw_images.csv")
metadata = pd.read_csv("/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/metadata_v2/metadata.csv", usecols=["Image"])
metadata.Image = metadata.Image.str.replace("/chip.npy", "", regex=False)
metadata.rename({"Image": "image"}, axis=1, inplace=True)
# patches = ['18NTN', '18NTP', '18NUL', '18NUM', '18NUN', '18NUP']
# metadata = metadata.query("patch in ['18NTN', '18NTP', '18NUL', '18NUM', '18NUN', '18NUP']")
# metadata = metadata.query("patch in ['18NTN', '18NTP', '18NUL']")
metadata

In [ ]:
def get_elevation_map(path_to_metadata, path_to_save_map):

    corners = pd.read_pickle(path_to_metadata)["corners"]
    west, north = corners["nw"][::-1]
    east, south = corners["se"][::-1]

    elevation.clip(bounds=(west, south, east, north), output=f"{path_to_save_map}", margin="1%")
    elevation.clean()

    clear_output()

In [ ]:
path_to_metadata = "/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Dataset/18NVK/18NVK_0_1/(0, 0)/metadata.pkl"
path_to_save_map =  f"{path_to_data}/Elevations/elevation.tif"
get_elevation_map(path_to_metadata, path_to_save_map)

In [ ]:
path = "/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Dataset/18NTN/18NTN_8_5/(0, 400)/metadata.pkl"
save_path = f"{path_to_data}/Elevations/elevation.tif"
#get_elevation_map(path, save_path)

In [ ]:
EDA.read_geotiff_image("/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Elevations/elevation.tif").shape

In [ ]:
letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
letters_map=[{l:i for i, l in enumerate(letters)}][0]

In [ ]:
def generate_elevation_maps(metadata):

    metadata_elevations = []
    for path in tqdm(metadata.image):
        print(path)
        if not os.path.isdir(f"{path_to_data}/Elevations/{path}"):
            os.makedirs(f"{path_to_data}/Elevations/{path}")

        get_elevation_map(f"{path_to_data}/Dataset/{path}/metadata.pkl", f"{path_to_data}/Elevations/elevation.tif")

        if not os.path.isdir(f"{path_to_data}/sentinel_geo_images/{path}"):
            os.makedirs(f"{path_to_data}/sentinel_geo_images/{path}")

        #img = read_numpy_image(f"{path_to_data}/Dataset/{path}/chip.npy")
        #from_array_to_geotiff(f"{path_to_data}/sentinel_geo_images/{path}/chip.tif", img[:3], f"{path_to_data}/Dataset/{path}/metadata.pkl", crs=3116)

        shutil.move(f"{path_to_data}/Elevations/elevation.tif", f"{path_to_data}/Elevations/{path}/elevation.tif")

        id_number = re.sub(r"[a-zA-Z_\(\)\,\\\/\s+]", "", path)
        id_letter = re.sub(r"[\d_\(\)\,\\\/\s+]", "", path)
        id_letter = "".join(str(letters_map[l]) for l in id_letter[:3])
        id = int(id_number + id_letter)
        metadata_elevations.append({"Id":id, "Elevation":f"{path}/elevation.tif", "Image":f"{path}/chip.npy"})
        

    return pd.DataFrame(metadata_elevations)

In [ ]:
metadata_elevations = generate_elevation_maps(metadata)